In [8]:
import os
import pandas as pd


def stock_or_not(str_code):
    """
    Check whether str_code stands for a stock.
    :param str_code: 6-digit string
    :return: bool
    """
    str_code = str_code.zfill(6)
    if str_code[:2] in ['600', '601', '603', '688', '689']:  # 未考虑B股
        return True
    elif str_code[:1] in ['00', '30']:  # 未考虑B股(20,与204冲突)
        return True
    else:
        return False


def ce_or_not(str_code):
    """
    Check whether str_code stands for cash equivalent.
    :param str_code: 6-digit string
    :return: bool
    """
    str_code = str_code.zfill(6)
    if str_code[:2] in ['204', '511', '159', '519', '521', '660']:
        return True
    elif str_code[:1] in ['13']:
        return True


def process_raw_data_cash(fpath_holding):
    str_ext = os.path.splitext(fpath_holding)[1]
    if str_ext in ['.xlsx', '.xls']:
        df_capital = pd.read_excel(fpath_holding, nrows=1)
        df_holding = pd.read_excel(fpath_holding, skiprows=3)
    elif str_ext == '.csv':
        df_capital = pd.read_csv(fpath_holding, nrows=1, encoding='gbk',
                                 dtype={'资产账户': str, '总资产': float, '总负债': float, '净资产': float,
                                        '资金可用金': float})
        df_holding = pd.read_csv(fpath_holding, skiprows=3, encoding='gbk',
                                 dtype={'证券代码': str, '市值': float})
    else:
        raise TypeError('Unknown file type!')
    print(df_capital)
    print(df_holding)



In [10]:
str_date = '20200413'
fpath = f'D:/data/A_trading_data/1500+/A_result/{str_date}/2hao_MS/holding_sw.csv'
process_raw_data_cash(fpath)

         资产账户          总资产           市值        资金资产       资金可用金       资金冻结金  \
0  2019029790  45315485.21  43197535.07  2117950.14  2117950.14  3991085.04   

        资金解冻金 货币代码  Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  \
0  4271672.05  人民币         NaN         NaN          NaN          NaN   

   Unnamed: 12  
0          NaN  
    记录号    证券代码  证券名称   可用数量   证券数量     成本价     最新价        盈亏        市值  \
0     1  603360  百傲化学  12600  12600   29.79   30.99  15182.15  390474.0   
1     2  603528  多伦科技   5000   5000    7.81    7.83     93.34   39150.0   
2     3  603538   美诺华  10500  10500   35.30   35.35    557.04  371175.0   
3     4  603669  灵康药业  36200  36200   10.51   11.20  25083.33  405440.0   
4     5  603686  龙马环卫  16200  16200   15.17   15.13   -627.76  245106.0   
..  ...     ...   ...    ...    ...     ...     ...       ...       ...   
57   58  603069  海汽集团      0   6500    9.85    9.75   -618.89   63375.0   
58   59  002332  仙琚制药      0  24900   14.97   14.89  -1882.89 